In [ ]:
%matplotlib inline


# Mass Balancing

In the examples so far, Flowsheet objects are created by math operations, so they inherently balance.
A common problem in Mineral Processing (Metallurgy/Chemical/Process Engineering) is mass (or metallurgical) balancing.
When auditing a processing plant the data is collected by measurement and sampling/assaying. This data will never
balance of course due to sampling and measurement errors.

There exists a fundamental optimisation process that can balance the overall mass and components across a
system (network/flowsheet).  This example demonstrates that functionality.


In [ ]:
import logging
from functools import partial
from typing import Dict

import numpy as np
import pandas as pd
import plotly

from elphick.mass_composition import MassComposition
from elphick.mass_composition.balance import MCBalance
from elphick.mass_composition.flowsheet import Flowsheet
from elphick.mass_composition.utils.partition import napier_munn
from elphick.mass_composition.datasets.sample_data import size_by_assay

In [ ]:
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(module)s - %(funcName)s: %(message)s',
                    datefmt='%Y-%m-%dT%H:%M:%S%z')

## Create a MassComposition object

We get some demo data in the form of a pandas DataFrame



In [ ]:
df_data: pd.DataFrame = size_by_assay()
df_data

Create the object



In [ ]:
mc_size: MassComposition = MassComposition(df_data, name='size sample')
print(mc_size)
mc_size.aggregate()

We partially initialise a partition function
The dim argument is added to inform the split method which dimension to apply the function/split to



In [ ]:
partition = partial(napier_munn, d50=0.150, ep=0.1, dim='size')

## Create a Network that balances

Separate the object using the defined partition



In [ ]:
mc_coarse, mc_fine = mc_size.split_by_partition(partition_definition=partition, name_1='coarse', name_2='fine')

fs: Flowsheet = Flowsheet().from_streams([mc_size, mc_coarse, mc_fine])
print(fs.balanced)

fig = fs.table_plot(plot_type='network', table_pos='left', table_area=0.3)
fig

Demonstrate that the data balances with the balance plot



In [ ]:
fig = fs.plot_balance()
# noinspection PyTypeChecker
plotly.io.show(fig)  # this call to show will set the thumbnail for the gallery

The balance plot can be colored by a specified column or index/dimension.



In [ ]:
fig = fs.plot_balance(color='size')
fig

## Create an imbalanced network

Modify one stream to corrupt the balance



In [ ]:
df_coarse_2 = mc_coarse.data.to_dataframe().apply(lambda x: np.random.normal(loc=x, scale=np.std(x)))
mc_coarse_2: MassComposition = MassComposition(data=df_coarse_2, name='coarse')
mc_coarse_2 = mc_coarse_2.set_parent_node(mc_size)

# create a new network - which does not balance
fs_ub: Flowsheet = Flowsheet().from_streams([mc_size, mc_coarse_2, mc_fine])
print(fs_ub.balanced)

fig = fs_ub.table_plot(plot_type='network', table_pos='left', table_area=0.3)
fig

In [ ]:
fig = fs_ub.plot_balance()
fig

## Balance the Flowsheet



..  note::

    This example has not yet been completed...



In [ ]:
mcb: MCBalance = MCBalance(fs=fs_ub)

# SD configuration
# df_sds: pd.DataFrame = mcb.create_balance_config(best_measurements='input')

# cost functions
cfs: Dict = mcb._create_cost_functions()
# check for a zero cost when passing the measured values
for k, v in cfs.items():
    x = mcb.fs.to_dataframe().loc[k, :].drop(columns=['mass_wet']).values.ravel()
    y = v(x=x)
    print(k, y)

df_bal: pd.DataFrame = mcb.optimise()

# create a network using the balanced data
fs_bal: Flowsheet = Flowsheet.from_dataframe(df=df_bal, name='balanced', mc_name_col='name')
fig = fs_bal.plot_parallel(color='name')
fig.show()